#  **Assignment-AI-NLP Intern-Emplay Inc**

## **submitted by: Aditi Sharma**
**aditi0712sharma@gmail.com**

**Installing the required dependencies**

In [4]:
!pip install -q transformers
!pip install -q requests
!pip install -q rouge
!pip install -q nltk rouge 
!pip install -q textstat 
!pip install -q nltk

In [5]:
import nltk

In [6]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /usr/share/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /usr/share/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to /usr/share/nltk_data...
[nltk_data]    | Downloading pa

True

**Importing the required modules**

In [7]:
import re, requests, numpy as np, pandas as pd
from bs4 import BeautifulSoup
from transformers import pipeline
from nltk.translate import bleu_score
from rouge import Rouge
import textstat, random
from nltk.translate.bleu_score import SmoothingFunction
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

**Scraping the article from [Wikipedia](http://en.wikipedia.org/wiki/Alexander_the_Great)**

In [8]:
wiki_url = "https://en.wikipedia.org/wiki/Alexander_the_Great"

##### *Scraping the desired text*

In [9]:
def scrape_wikipedia(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Initialize a dictionary to store the content for each heading and subheading
    content_dict = {}
    current_heading = None
    current_subheading = None
    current_content = ''

    # Iterate through elements
    for element in soup.find_all(['h2', 'h3', 'p']):
        if element.name == 'h2':
            # Save content for the previous subheading
            if current_subheading and current_content:
                content_dict[(current_heading, current_subheading)] = current_content

            # Update current heading and reset content
            current_heading = element.text
            current_content = ''
            
        elif element.name == 'h3':
            # Save content for the previous subheading
            if current_subheading and current_content:
                content_dict[(current_heading, current_subheading)] = current_content

            # Update current subheading and reset content
            current_subheading = element.text
            current_content = ''
        elif element.name == 'p' and current_heading and current_subheading:
            # Add paragraphs to the current content
            current_content += ' ' + element.text

    # Add content for the last subheading
    if current_heading and current_subheading and current_content:
        content_dict[(current_heading, current_subheading)] = current_content

    return content_dict

In [10]:
content_dict = scrape_wikipedia(wiki_url)
#print(content_dict)

##### Summarize the content using Pre-Trained 

Funtion for Summarization

In [11]:
def summarize_text(text):
    summarizer = pipeline('summarization', model='facebook/bart-large-cnn')
    summary = summarizer(text, max_length=100, min_length=20, length_penalty=0.5, num_beams=2, early_stopping=True)[0]['summary_text']
    return summary

> Create an outputfile into which the whole summarized article will be stored.

In [12]:
output_file = "final_output_summary.txt"

#### Summarize content for each heading and subheading and store it in a file.

In [13]:
with open(output_file, "w", encoding="utf-8") as file:
    for (heading, subheading), content in content_dict.items():
        # Write heading only when a new heading is encountered
        file.write(f"\n\n{heading}\n")
        
        # Write subheading
        file.write(f"{subheading}\n")

        # Check if content inside the subheading is empty
        if not content.strip():
            file.write("No content available for this subheading\n")
            continue

        # Break down the content into chunks of 1024 tokens
        chunk_size = 1024
        chunks = [content[i:i + chunk_size] for i in range(0, len(content), chunk_size)]

            # Summarize each chunk
        for i, chunk in enumerate(chunks):
            # Summarize the chunk
            summary = summarize_text(chunk)

            # Write the summary to the file
            file.write(summary + ' ')

# Output file created with the summaries
print(f"Summaries saved to {output_file}")

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Your max_length is set to 100, but your input_length is only 44. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=22)
Your max_length is set to 100, but your input_length is only 22. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=11)
Your max_length is set to 100, but your input_length is only 85. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=42)
Your max_length is set to 100, but your input_length is only 65. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=32)
Your

Summaries saved to final_output_summary.txt


### Evaluation of the model

> Creating a Dataframe to store the metrics

In [14]:
columns = ["Metric", "Average Score"]
metrics_df = pd.DataFrame(columns=columns)

> Function to calculate the Evaluation Metrics

In [15]:
def calculate_metric(metric_name, content, metric_function):
    # Check if content is empty
    if not content.strip():
        print("No content available")
        return None

    # Combine all sentences into a single summary
    full_summary = summarize_text(content)

    # Calculate the metric
    smoothing = SmoothingFunction().method1
    metric_value = metric_function(full_summary)

    # Add metric to the DataFrame
    metrics_df.loc[len(metrics_df)] = [metric_name, metric_value]

> List of metrics and their corresponding functions

In [16]:
evaluation_metrics = [
    ("BLEU Score", lambda x: bleu_score.sentence_bleu([nltk.sent_tokenize(x)], nltk.sent_tokenize(x))),
    ("ROUGE F1 Score", lambda x: sum(score["rouge-1"]["f"] for score in Rouge().get_scores(x, x)) / len(Rouge().get_scores(x, x))),
    ("Flesch Reading Ease", textstat.flesch_reading_ease),
    ("Average Sentence Length", textstat.avg_sentence_length),
    ("Average Word Length", textstat.avg_letter_per_word),
    ("Cosine Similarity", lambda x: cosine_similarity(TfidfVectorizer().fit_transform([x, x]))[0, 0]),
]

> Generate random paragraphs for evaluation for faster checking

In [17]:
def generate_random_paragraph(content_dict, num_paragraphs=1):
    random_paragraphs = []
    for _ in range(num_paragraphs):
        # Select a random content from content_dict
        random_heading, random_subheading = random.choice(list(content_dict.keys()))
        random_content = content_dict[(random_heading, random_subheading)]
        random_paragraphs.append(random_content)
    return random_paragraphs

In [18]:
random_paragraph = generate_random_paragraph(content_dict)

In [19]:
random_paragraph

[' When Alexander set out for Asia, he left his general Antipater, an experienced military and political leader and part of Philip II\'s "Old Guard", in charge of Macedon.[62] Alexander\'s sacking of Thebes ensured that Greece remained quiet during his absence.[62] The one exception was a call to arms by Spartan king Agis III in 331\xa0BC, whom Antipater defeated and killed in the battle of Megalopolis.[62] Antipater referred the Spartans\' punishment to the League of Corinth, which then deferred to Alexander, who chose to pardon them.[111] There was also considerable friction between Antipater and Olympias, and each complained to Alexander about the other.[112]\n In general, Greece enjoyed a period of peace and prosperity during Alexander\'s campaign in Asia.[113] Alexander sent back vast sums from his conquest, which stimulated the economy and increased trade across his empire.[114] However, Alexander\'s constant demands for troops and the migration of Macedonians throughout his empi

> Calculating the metrics 

In [20]:
for content in random_paragraph:
    for metric_name, metric_function in evaluation_metrics:
        calculate_metric(metric_name, content, metric_function)

**The Generated Metrics for Evaluation:**

In [21]:
metrics_df = metrics_df.groupby("Metric").mean().reset_index()
print(metrics_df)

                    Metric  Average Score
0  Average Sentence Length          13.30
1      Average Word Length           5.47
2               BLEU Score           1.00
3        Cosine Similarity           1.00
4      Flesch Reading Ease          49.52
5           ROUGE F1 Score           1.00
